# ML Trading Pipeline - Unified Overlay Implementation

## Overview
This notebook implements the unified overlay architecture for MetaStackerBandit with **one-to-one parity** to the original Bandit New notebook. Instead of training separate models for 5m, 1h, and 12h timeframes, we train a single model on 5-minute data that can generate signals for all timeframes using rollup overlays.

## Key Changes from Original:
1. **Single Model Training**: One model trained on 5m data for all timeframes
2. **Overlay Feature Engineering**: Rollup features for 15m and 1h from 5m base
3. **Unified Feature Schema**: Same features across all timeframes
4. **Multi-Timeframe Signals**: Generate signals for 5m, 15m, 1h simultaneously
5. **Enhanced Bandit**: Multi-level bandit for timeframe and signal selection

## Structure (Matching Original):
1. Import Required Libraries
2. Configuration & Parameters
3. Data Loading and Processing
4. Feature Engineering (Unified)
5. Trading Signal Generation (Multi-Arm Bandit)
6. Backtesting Engine
7. Grid Search Optimization
8. Results Analysis & Visualization
9. Export & Deployment


In [1]:
# Imports (Matching Original Bandit New)
import pandas as pd, numpy as np, warnings, os, json, joblib
from datetime import datetime, timedelta
from collections import defaultdict, deque
import scipy.stats as stats
from scipy.optimize import minimize_scalar
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier, HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNetCV, HuberRegressor, LogisticRegression
from sklearn.model_selection import KFold, TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score, classification_report, confusion_matrix, log_loss
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.base import clone
from typing import Dict, List, Tuple, Optional, Union
import random
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

# Deterministic seeds
np.random.seed(42)
random.seed(42)

print("Imports loaded successfully!")


Imports loaded successfully!


In [ ]:
# Single-source threshold & cost config for allocator-only mode (Matching Original)
# These globals are consumed by consolidated arms and backtest cells.
try:
    S_MIN
except NameError:
    S_MIN = 0.12
try:
    M_MIN
except NameError:
    M_MIN = 0.12
try:
    CONF_MIN
except NameError:
    CONF_MIN = 0.60
try:
    ALPHA_MIN
except NameError:
    ALPHA_MIN = 0.10
try:
    COOLDOWN
except NameError:
    COOLDOWN = 1
try:
    COST_BP
except NameError:
    COST_BP = 5.0
try:
    IMPACT_K
except NameError:
    IMPACT_K = 0.0

# Risk and execution controls (Matching Original)
try:
    SIGMA_TARGET
except NameError:
    SIGMA_TARGET = 0.20  # per-bar target scaler proxy
try:
    POS_MAX
except NameError:
    POS_MAX = 1.0
try:
    DD_STOP
except NameError:
    DD_STOP = 0.05
try:
    LATENCY_BARS
except NameError:
    LATENCY_BARS = 0
try:
    SLIPPAGE_BPS
except NameError:
    SLIPPAGE_BPS = 0.0
try:
    COST_CONVENTION
except NameError:
    COST_CONVENTION = 'per_transition'  # or 'per_roundtrip'
try:
    SMOOTH_BETA
except NameError:
    SMOOTH_BETA = 0.0

# Overlay-specific parameters (NEW)
OVERLAY_TIMEFRAMES = ["5m", "15m", "1h"]
ROLLUP_WINDOWS = {"15m": 3, "1h": 12}  # 3x5m=15m, 12x5m=1h
TIMEFRAME_WEIGHTS = {"5m": 0.5, "15m": 0.3, "1h": 0.2}

# Feature engineering parameters (unified across timeframes)
MOMENTUM_PERIODS = [1, 3, 6]  # Consistent momentum periods
EMA_PERIOD = 20  # Consistent EMA period
ROLLING_WINDOW = 100  # Consistent rolling window
RV_WINDOWS = {"1h": 12, "15m": 3, "1d": 288}  # RV windows

print("Configuration loaded:")
print(f"  Signal thresholds: S_MIN={S_MIN}, M_MIN={M_MIN}, CONF_MIN={CONF_MIN}, ALPHA_MIN={ALPHA_MIN}")
print(f"  Risk controls: SIGMA_TARGET={SIGMA_TARGET}, POS_MAX={POS_MAX}, DD_STOP={DD_STOP}")
print(f"  Overlay timeframes: {OVERLAY_TIMEFRAMES}")
print(f"  Rollup windows: {ROLLUP_WINDOWS}")
print(f"  Timeframe weights: {TIMEFRAME_WEIGHTS}")


Configuration loaded:
  Signal thresholds: S_MIN=0.12, M_MIN=0.12, CONF_MIN=0.6, ALPHA_MIN=0.1
  Risk controls: SIGMA_TARGET=0.2, POS_MAX=1.0, DD_STOP=0.05
  Overlay timeframes: ['5m', '15m', '1h']
  Rollup windows: {'15m': 3, '1h': 12}
  Timeframe weights: {'5m': 0.5, '15m': 0.3, '1h': 0.2}


In [3]:
# Load data (Matching Original Structure)
print("Loading data...")

# Load OHLCV data
df = pd.read_csv('ohlc_btc_5m.csv') if os.path.exists('ohlc_btc_5m.csv') else pd.DataFrame()
if not df.empty:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    print(f"Loaded {len(df)} 5m bars")
else:
    print("Error: No OHLCV data found")

# Load funding data
funding_df = pd.read_csv('funding_btc.csv') if os.path.exists('funding_btc.csv') else pd.DataFrame()
if not funding_df.empty:
    funding_df['timestamp'] = pd.to_datetime(funding_df['timestamp'])
    funding_df = funding_df.sort_values('timestamp').reset_index(drop=True)
    print(f"Loaded {len(funding_df)} funding records")

# Load cohort data
cohort_top = pd.read_csv('top_cohort.csv') if os.path.exists('top_cohort.csv') else pd.DataFrame()
cohort_bot = pd.read_csv('bottom_cohort.csv') if os.path.exists('bottom_cohort.csv') else pd.DataFrame()

# Process cohort addresses (Matching Original)
for cdf in (cohort_top, cohort_bot):
    if not cdf.empty and 'user' not in cdf.columns:
        col = next((c for c in ['Account', 'address', 'user', 'addr'] if c in cdf.columns), None)
        if col: cdf['user'] = cdf[col].astype(str)

cohort_top_users = set(cohort_top.get('user', pd.Series(dtype=str)).dropna().astype(str))
cohort_bot_users = set(cohort_bot.get('user', pd.Series(dtype=str)).dropna().astype(str))
cohort_addresses = cohort_top_users | cohort_bot_users

print(f"Cohort addresses: {len(cohort_addresses)}")

# Load fills data (Matching Original Processing)
fills = pd.read_csv('historical_trades_btc.csv', low_memory=False) if os.path.exists('historical_trades_btc.csv') else pd.DataFrame()
if not fills.empty:
    if 'user' not in fills.columns:
        if 'Account' in fills.columns: fills['user'] = fills['Account'].astype(str)
    
    # Parse timestamps (Matching Original)
    ts_col = 'Timestamp' if 'Timestamp' in fills.columns else (next((c for c in ['timestamp', 'ts'] if c in fills.columns), None))
    ts_raw = pd.to_numeric(fills[ts_col], errors='coerce') if ts_col else pd.Series(dtype='float64')
    if ts_raw.notna().any():
        med = ts_raw.dropna().median()
        unit = 'ns' if med>1e14 else ('ms' if med>1e12 else 's')
        fills['timestamp'] = pd.to_datetime(ts_raw, unit=unit, errors='coerce')
    else:
        time_col = next((c for c in ['Timestamp IST', 'time'] if c in fills.columns), None)
        fills['timestamp'] = pd.to_datetime(fills[time_col], errors='coerce') if time_col else pd.NaT
    
    # Map helpers (Matching Original)
    if 'Execution Price' in fills.columns and 'px' not in fills.columns:
        fills['px'] = pd.to_numeric(fills['Execution Price'], errors='coerce')
    if 'Size Tokens' in fills.columns and 'sz' not in fills.columns:
        fills['sz'] = pd.to_numeric(fills['Size Tokens'], errors='coerce')
    if 'Size USD' in fills.columns and 'notional' not in fills.columns:
        fills['notional'] = pd.to_numeric(fills['Size USD'], errors='coerce')
    if 'Side' in fills.columns and 'side' not in fills.columns:
        fills['side'] = fills['Side'].map({'BUY':'B','SELL':'A'}).fillna(fills['Side'].astype(str))
    
    print(f"Loaded {len(fills)} fills")

print("Data loading complete!")


Loading data...
Loaded 51840 5m bars
Loaded 4320 funding records
Cohort addresses: 174
Loaded 1038195 fills
Data loading complete!


In [4]:
# Unified Feature Engineering (Matching Original + Overlay)
print("Creating unified features...")

# Create base features from 5m data
features_df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']].copy()

# Price and returns (Matching Original)
features_df['price'] = features_df['close']
features_df['returns'] = features_df['price'].pct_change()

# Technical indicators (unified parameters)
features_df['hl_range'] = features_df['high'] - features_df['low']
features_df['oc_range'] = np.abs(features_df['open'] - features_df['close'])
features_df['typical_price'] = (features_df['high'] + features_df['low'] + features_df['close']) / 3
features_df['true_range'] = np.maximum(
    features_df['high'] - features_df['low'],
    np.maximum(
        np.abs(features_df['high'] - features_df['close'].shift(1)),
        np.abs(features_df['low'] - features_df['close'].shift(1))
    )
)

# Momentum features (consistent periods)
for h in MOMENTUM_PERIODS:
    features_df[f'mom_{h}'] = features_df['price'].pct_change(periods=h)

# EMA and mean reversion (unified parameters)
features_df['ema20'] = features_df['price'].ewm(span=EMA_PERIOD, adjust=False).mean()
features_df['mr_ema20'] = (features_df['price'] - features_df['ema20']) / features_df['ema20']

# Z-score normalization (unified rolling window)
rolling_mean = features_df['price'].rolling(window=ROLLING_WINDOW, min_periods=20).mean()
rolling_std = features_df['price'].rolling(window=ROLLING_WINDOW, min_periods=20).std()
features_df['mr_ema20_z'] = (features_df['price'] - rolling_mean) / (rolling_std + 1e-8)

# Volatility features (unified RV windows)
features_df['rv_1h'] = features_df['returns'].rolling(window=RV_WINDOWS['1h'], min_periods=6).apply(lambda x: (x**2).sum())
features_df['rv_15m'] = features_df['returns'].rolling(window=RV_WINDOWS['15m'], min_periods=2).apply(lambda x: (x**2).sum())
features_df['rv_1d'] = features_df['returns'].rolling(window=RV_WINDOWS['1d'], min_periods=50).apply(lambda x: (x**2).sum())

# Volatility regime
rv_threshold = features_df['rv_1h'].rolling(window=100, min_periods=25).quantile(0.75)
features_df['regime_high_vol'] = (features_df['rv_1h'] > rv_threshold).astype(int)

# Enhanced volatility features (Matching Original)
if {'high','low','close','open'}.issubset(features_df.columns):
    gk_vol = np.log(features_df['high']/features_df['low'])**2 / 2 - (2*np.log(2)-1) * np.log(features_df['close']/features_df['open'])**2
    features_df['gk_volatility'] = gk_vol.rolling(12, min_periods=6).mean()
    
    parkinson_vol = np.log(features_df['high']/features_df['low'])**2 / (4 * np.log(2))
    features_df['parkinson_volatility'] = parkinson_vol.rolling(12, min_periods=6).mean()
    
    vol_percentile_20 = features_df['gk_volatility'].rolling(100, min_periods=50).quantile(0.2)
    vol_percentile_80 = features_df['gk_volatility'].rolling(100, min_periods=50).quantile(0.8)
    features_df['vol_regime_low'] = (features_df['gk_volatility'] <= vol_percentile_20).astype('float')
    features_df['vol_regime_high'] = (features_df['gk_volatility'] >= vol_percentile_80).astype('float')

# Jump detection (Matching Original)
return_std = features_df['returns'].rolling(50, min_periods=25).std()
features_df['jump_indicator'] = (np.abs(features_df['returns']) > 3 * return_std).astype('float')
features_df['jump_magnitude'] = np.abs(features_df['returns']) / (return_std + 1e-8)

# Volume features (Matching Original)
features_df['volume_intensity'] = features_df['volume'] / features_df['volume'].rolling(50, min_periods=25).mean()
features_df['price_volume_corr'] = features_df['returns'].rolling(20, min_periods=10).corr(features_df['volume'].pct_change())

# Price efficiency (Matching Original)
features_df['price_efficiency'] = np.abs(features_df['close'] - features_df['open']) / (features_df['high'] - features_df['low'] + 1e-8)

# VWAP momentum (Matching Original)
features_df['vwap'] = (features_df['high'] + features_df['low'] + features_df['close']) / 3
features_df['vwap_momentum'] = features_df['vwap'].pct_change(periods=5)

# Depth proxy (simplified)
features_df['depth_proxy'] = features_df['volume'] / (features_df['hl_range'] + 1e-8)

# Funding features (if available)
if funding_df is not None and not funding_df.empty:
    funding_df['timestamp'] = pd.to_datetime(funding_df['timestamp'])
    features_df = features_df.merge(funding_df[['timestamp', 'funding_rate']], on='timestamp', how='left')
    features_df['funding_rate'] = features_df['funding_rate'].fillna(0.0)
    features_df['funding_momentum_1h'] = features_df['funding_rate'].rolling(12, min_periods=6).mean()
else:
    features_df['funding_rate'] = 0.0
    features_df['funding_momentum_1h'] = 0.0

# Flow features (placeholder - would need actual flow data)
features_df['flow_diff'] = 0.0

print(f"Created unified features: {features_df.shape}")
print(f"Feature columns: {list(features_df.columns)}")


Creating unified features...
Created unified features: (51840, 37)
Feature columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'price', 'returns', 'hl_range', 'oc_range', 'typical_price', 'true_range', 'mom_1', 'mom_3', 'mom_6', 'ema20', 'mr_ema20', 'mr_ema20_z', 'rv_1h', 'rv_15m', 'rv_1d', 'regime_high_vol', 'gk_volatility', 'parkinson_volatility', 'vol_regime_low', 'vol_regime_high', 'jump_indicator', 'jump_magnitude', 'volume_intensity', 'price_volume_corr', 'price_efficiency', 'vwap', 'vwap_momentum', 'depth_proxy', 'funding_rate', 'funding_momentum_1h', 'flow_diff']


In [5]:
# Create Overlay Features (NEW - Overlay Architecture)
print("Creating overlay features...")

def create_overlay_features(base_features_df, timeframe, rollup_window):
    """Create overlay features by rolling up base 5m features"""
    if timeframe == "5m":
        return base_features_df.copy()
    
    # Create rollup bars
    rollup_df = base_features_df.copy()
    rollup_df['rollup_group'] = rollup_df.index // rollup_window
    
    # Get all available columns for aggregation
    agg_dict = {
        'timestamp': 'last',
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'price': 'last',
        'returns': lambda x: (x.iloc[-1] / x.iloc[0] - 1) if len(x) > 1 else 0,
        'mom_1': 'last',
        'mom_3': 'last', 
        'mom_6': 'last',
        'mr_ema20_z': 'last',
        'rv_1h': 'mean',
        'regime_high_vol': 'mean',
        'gk_volatility': 'mean',
        'jump_magnitude': 'mean',
        'volume_intensity': 'mean',
        'price_efficiency': 'mean',
        'vwap_momentum': 'last',
        'depth_proxy': 'mean',
        'funding_rate': 'mean',
        'funding_momentum_1h': 'mean',
        'flow_diff': 'sum'
    }
    
    # Add price_volume_corr if it exists
    if 'price_volume_corr' in rollup_df.columns:
        agg_dict['price_volume_corr'] = 'mean'
    
    overlay_features = rollup_df.groupby('rollup_group').agg(agg_dict).reset_index(drop=True)
    
    # Recalculate some features for the new timeframe
    overlay_features['price'] = overlay_features['close']
    overlay_features['hl_range'] = overlay_features['high'] - overlay_features['low']
    overlay_features['typical_price'] = (overlay_features['high'] + overlay_features['low'] + overlay_features['close']) / 3
    
    return overlay_features

# Create overlay features for all timeframes
overlay_features = {}
overlay_features['5m'] = create_overlay_features(features_df, '5m', 1)
overlay_features['15m'] = create_overlay_features(features_df, '15m', ROLLUP_WINDOWS['15m'])
overlay_features['1h'] = create_overlay_features(features_df, '1h', ROLLUP_WINDOWS['1h'])

print(f"Overlay features created:")
for tf, features in overlay_features.items():
    print(f"  {tf}: {features.shape[0]} bars")

# Store base features for backtesting
bt = features_df.copy()  # Matching original variable name
bt['returns'] = bt['price'].pct_change()  # Ensure returns column exists


Creating overlay features...
Overlay features created:
  5m: 51840 bars
  15m: 17280 bars
  1h: 4320 bars


In [6]:
# Bandit allocator: class and backtest function (Matching Original)
import numpy as np
import pandas as pd
from typing import Tuple, Optional

class SimpleThompsonBandit:
    def __init__(self, n_arms: int):
        self.counts = np.zeros(n_arms, dtype=float)
        self.means = np.zeros(n_arms, dtype=float)
        self.vars = np.ones(n_arms, dtype=float)
        
    def select(self, eligible: np.ndarray) -> int:
        """Select arm using Thompson Sampling"""
        if not np.any(eligible):
            return 0
        
        # Sample from posterior
        samples = np.random.normal(self.means, np.sqrt(self.vars))
        
        # Only consider eligible arms
        samples[~eligible] = -np.inf
        
        return int(np.argmax(samples))
    
    def update(self, arm: int, reward: float):
        """Update arm statistics with new reward"""
        self.counts[arm] += 1
        n = self.counts[arm]
        
        # Update mean
        self.means[arm] = (self.means[arm] * (n - 1) + reward) / n
        
        # Update variance (simplified)
        if n > 1:
            self.vars[arm] = 1.0 / n

print("Bandit class defined")


Bandit class defined


In [7]:
# Comprehensive Backtesting Engine (Matching Original)
def run_allocator_backtest(
    bt_df: pd.DataFrame,
    arm_signals: np.ndarray,
    arm_eligible: np.ndarray,
    adv_series: pd.Series,
    cooldown_bars: int,
    cost_bps: float,
    impact_k: float,
    side_eps_vec: np.ndarray,
    eps: float = 1e-12,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, Dict]:
    """Comprehensive backtesting with bandit allocation (Matching Original)"""
    
    n = len(bt_df)
    rets = bt_df['returns'].values if 'returns' in bt_df.columns else np.zeros(n)
    
    # ADV handling (Matching Original)
    adv_valid = isinstance(adv_series, pd.Series) and not adv_series.dropna().empty
    adv_arr = adv_series.reindex(bt_df.index).to_numpy() if adv_valid else np.ones(n, dtype=float)
    impact_k_eff = impact_k if adv_valid else 0.0
    
    bandit = SimpleThompsonBandit(n_arms=arm_signals.shape[1])
    
    pos = 0.0
    pos_smooth = 0.0
    last_flip_idx = -10**9
    
    exec_pos_buffer = deque(maxlen=cooldown_bars + 1)
    exec_pos_buffer.append(0.0)
    
    records_eq = []
    records_tr = []
    records_bu = []
    
    cum_equity = 1.0
    equity_series = np.ones(n, dtype=float)
    
    sigma_target = float(globals().get('SIGMA_TARGET', 0.20))
    pos_max = float(globals().get('POS_MAX', 1.0))
    dd_stop = float(globals().get('DD_STOP', 0.05))
    latency_k = int(globals().get('LATENCY_BARS', 0))
    
    for t in range(n):
        exec_pos = exec_pos_buffer[0] if latency_k > 0 else pos_smooth
        
        # Check eligibility and select arm (Matching Original)
        elig = arm_eligible[t] if t < len(arm_eligible) else np.array([False]*arm_signals.shape[1])
        desired_side = pos
        chosen = None
        
        if np.any(elig):
            chosen = bandit.select(elig)
            raw_val = float(arm_signals[t, chosen])
            th = float(side_eps_vec[chosen]) if (side_eps_vec is not None) else 0.0
            
            if abs(raw_val) < th:
                desired_side = 0.0
            else:
                desired_side = np.sign(raw_val) * pos_max
        
        # Position sizing with volatility targeting (Matching Original)
        if t > 0 and abs(rets[t]) > eps:
            vol_est = np.std(rets[max(0, t-20):t+1])
            if vol_est > eps:
                vol_scaler = sigma_target / (vol_est * np.sqrt(252 * 24 * 12))  # Annualized
                desired_side *= min(vol_scaler, 2.0)  # Cap at 2x
        
        # Cooldown logic (Matching Original)
        if abs(desired_side - exec_pos) > eps and (t - last_flip_idx) >= cooldown_bars:
            exec_pos_buffer.append(desired_side)
            last_exec_pos = exec_pos
            
            # Cost calculation (Matching Original)
            cost_bps_eff = cost_bps
            if impact_k_eff > 0 and adv_valid:
                notional_change = abs(desired_side - exec_pos)
                impact_bps = impact_k_eff * notional_change / (adv_arr[t] + eps)
                cost_bps_eff += impact_bps
            
            records_tr.append((bt_df.index[t], exec_pos, desired_side, cost_bps_eff))
            last_exec_pos = exec_pos
            last_flip_idx = t
        
        pnl = rets[t] * (exec_pos_buffer[0] if latency_k > 0 else exec_pos)
        pnl -= (cost_bps / 10000.0) if cost_bps > 0 else 0.0
        cum_equity *= (1.0 + pnl)
        equity_series[t] = cum_equity
        records_eq.append((bt_df.index[t], cum_equity))
        
        if chosen is not None:
            bandit.update(chosen, pnl)
            records_bu.append((bt_df.index[t], int(chosen), float(pnl)))
        
        # Drawdown stop (Matching Original)
        if dd_stop > 0:
            peak = np.max(equity_series[:t+1])
            if peak > 0 and (cum_equity / peak - 1.0) < -dd_stop:
                exec_pos_buffer.append(0.0)
                last_exec_pos = 0.0
        
        pos_smooth = exec_pos_buffer[0] if exec_pos_buffer else 0.0
    
    Eq = pd.DataFrame.from_records(records_eq, columns=['ts', 'equity']).set_index('ts')
    Tr = pd.DataFrame.from_records(records_tr, columns=['ts', 'from_pos', 'to_pos', 'cost_bps'])
    Bu = pd.DataFrame.from_records(records_bu, columns=['ts', 'chosen', 'reward'])
    
    # Calculate performance metrics (Matching Original)
    eq_rets = Eq['equity'].pct_change().dropna()
    annualizer = float(globals().get('ANNUALIZER', np.sqrt(365*24*12)))
    
    sharpe = float(annualizer * eq_rets.mean() / (eq_rets.std() if eq_rets.std() != 0 else np.nan)) if len(eq_rets) else np.nan
    
    # Sortino ratio (Matching Original)
    downside_rets = eq_rets[eq_rets < 0]
    sortino = float(annualizer * eq_rets.mean() / (downside_rets.std() if len(downside_rets) > 0 else np.nan)) if len(eq_rets) else np.nan
    
    maxdd = float(-(Eq['equity'] / Eq['equity'].cummax() - 1.0).min()) if not Eq.empty else np.nan
    
    # Turnover (Matching Original)
    turnover = float(np.abs(Tr['to_pos'].astype(float) - Tr['from_pos'].astype(float)).sum()) if not Tr.empty else 0.0
    
    # Hit rate (Matching Original)
    hit_rate = float((Tr['pnl_$'] > 0).sum() / max(len(Tr), 1)) if not Tr.empty and 'pnl_$' in Tr.columns else np.nan
    
    metrics = {
        'final_equity': float(Eq['equity'].iloc[-1]) if len(Eq) else 1.0,
        'n_trades': int(len(Tr)),
        'sharpe': sharpe,
        'sortino': sortino,
        'maxDD': maxdd,
        'turnover': turnover,
        'hit_rate': hit_rate,
    }
    
    return Eq, Tr, Bu, metrics

print("Backtesting engine defined")


Backtesting engine defined


In [8]:
# Model Training and Signal Generation (Matching Original + Overlay)
print("Training unified model...")

# Prepare training data for 5m (base timeframe)
def prepare_training_data(features_df, cohort_signals_df, target_horizon=1):
    """Prepare training data for the unified model (Matching Original)"""
    # Merge features with cohort signals
    training_df = features_df.merge(cohort_signals_df, on='timestamp', how='left')
    
    # Fill missing cohort signals
    training_df['S_top'] = training_df['S_top'].fillna(0.0)
    training_df['S_bot'] = training_df['S_bot'].fillna(0.0)
    
    # Create target variable (future returns)
    training_df['future_return'] = training_df['price'].pct_change(periods=target_horizon).shift(-target_horizon)
    
    # Create classification target (3-class: down, neutral, up)
    training_df['target'] = 1  # neutral
    training_df.loc[training_df['future_return'] > 0.001, 'target'] = 2  # up
    training_df.loc[training_df['future_return'] < -0.001, 'target'] = 0  # down
    
    # Select features for training (Matching Original)
    feature_columns = [
        'mom_1', 'mom_3', 'mom_6', 'mr_ema20_z', 'rv_1h', 'regime_high_vol',
        'gk_volatility', 'jump_magnitude', 'volume_intensity', 'price_efficiency',
        'price_volume_corr', 'vwap_momentum', 'depth_proxy', 'funding_rate',
        'funding_momentum_1h', 'flow_diff', 'S_top', 'S_bot'
    ]
    
    # Filter available features
    available_features = [col for col in feature_columns if col in training_df.columns]
    
    # Prepare training data
    X = training_df[available_features].fillna(0.0)
    y = training_df['target']
    
    # Remove rows with missing targets
    valid_mask = ~y.isna()
    X = X[valid_mask]
    y = y[valid_mask]
    
    return X, y, available_features

# Process cohort signals (Matching Original)
def process_cohort_signals(fills_df, cohort_top_users, cohort_bot_users):
    """Process cohort signals from fills data (Matching Original)"""
    if fills_df.empty:
        return pd.DataFrame()
    
    # Filter cohort fills
    cohort_fills = fills_df[fills_df['user'].isin(cohort_top_users | cohort_bot_users)].copy()
    
    if cohort_fills.empty:
        return pd.DataFrame()
    
    # Calculate signals (Matching Original)
    cohort_fills['side_numeric'] = cohort_fills['side'].map({'B': 1, 'A': -1, 'BUY': 1, 'SELL': -1}).fillna(0)
    cohort_fills['impact'] = cohort_fills['side_numeric'] * cohort_fills['notional']
    
    # Group by time windows (Matching Original)
    cohort_fills['time_window'] = pd.to_datetime(cohort_fills['timestamp']).dt.floor('5T')
    
    signals_df = cohort_fills.groupby('time_window').agg({
        'impact': 'sum',
        'notional': 'sum',
        'user': 'count'
    }).reset_index()
    
    signals_df.columns = ['timestamp', 'net_impact', 'total_notional', 'trade_count']
    
    # Calculate normalized signals (Matching Original)
    signals_df['S_top'] = signals_df['net_impact'] / (signals_df['total_notional'] + 1e-8)
    signals_df['S_bot'] = signals_df['net_impact'] / (signals_df['total_notional'] + 1e-8)
    
    return signals_df

# Process cohort signals
cohort_signals = process_cohort_signals(fills, cohort_top_users, cohort_bot_users)
print(f"Processed cohort signals: {cohort_signals.shape}")

# Train unified model on 5m data
X_train, y_train, feature_columns = prepare_training_data(overlay_features['5m'], cohort_signals)

print(f"Training data shape: {X_train.shape}")
print(f"Target distribution: {y_train.value_counts().to_dict()}")
print(f"Feature columns: {len(feature_columns)}")

# Train base models (Matching Original)
base_models = {
    'xgb': HistGradientBoostingClassifier(max_iter=100, learning_rate=0.1, max_depth=6, random_state=42),
    'hgb': HistGradientBoostingClassifier(max_iter=150, learning_rate=0.05, max_depth=8, random_state=42),
    'lasso': LogisticRegression(C=0.1, penalty='l1', solver='liblinear', random_state=42),
    'logit': LogisticRegression(C=1.0, penalty='l2', random_state=42)
}

# Train models
for name, model in base_models.items():
    model.fit(X_train, y_train)
    print(f"Trained {name} model")

# Create meta-model (Matching Original)
base_predictions = {}
for name, model in base_models.items():
    pred = model.predict_proba(X_train)
    base_predictions[name] = pred

meta_features = np.hstack([base_predictions[name] for name in base_models.keys()])
meta_model = LogisticRegression(C=1.0, random_state=42)
meta_model.fit(meta_features, y_train)

print(f"Model training complete: {len(feature_columns)} features")


Training unified model...
Processed cohort signals: (9819, 6)
Training data shape: (51840, 18)
Target distribution: {1: 39906, 2: 6043, 0: 5891}
Feature columns: 18
Trained xgb model
Trained hgb model
Trained lasso model
Trained logit model
Model training complete: 18 features


In [9]:
# Create Overlay Signals (NEW - Multi-Timeframe Signal Generation)
print("Creating overlay signals...")

def create_overlay_signals(unified_features, cohort_signals, base_models, meta_model, feature_columns):
    """Create multi-arm signals for overlay timeframes (Matching Original Structure)"""
    
    signals_data = []
    
    for timeframe, features_df in unified_features.items():
        print(f"Creating signals for {timeframe}...")
        
        # Merge with cohort signals
        merged_df = features_df.merge(cohort_signals, on='timestamp', how='left')
        merged_df['S_top'] = merged_df['S_top'].fillna(0.0)
        merged_df['S_bot'] = merged_df['S_bot'].fillna(0.0)
        
        # Prepare features
        X = merged_df[feature_columns].fillna(0.0)
        
        # Get model predictions (Matching Original)
        base_preds = {}
        for name, model in base_models.items():
            pred = model.predict_proba(X)
            base_preds[name] = pred
        
        meta_features = np.hstack([base_preds[name] for name in base_models.keys()])
        meta_pred = meta_model.predict_proba(meta_features)
        
        # Extract model signal (Matching Original)
        p_up = meta_pred[:, 2]  # up probability
        p_down = meta_pred[:, 0]  # down probability
        s_model = p_up - p_down
        
        # Create signals for this timeframe (Matching Original Structure)
        timeframe_signals = pd.DataFrame({
            'timestamp': merged_df['timestamp'],
            'timeframe': timeframe,
            'S_top': merged_df['S_top'],
            'S_bot': merged_df['S_bot'],
            'S_mood': merged_df['S_top'] + merged_df['S_bot'],  # Combined mood signal
            'S_model': s_model,
            'p_up': p_up,
            'p_down': p_down,
            'confidence': np.maximum(p_up, p_down),
            'alpha': np.abs(s_model)
        })
        
        signals_data.append(timeframe_signals)
    
    return pd.concat(signals_data, ignore_index=True)

# Create overlay signals
overlay_signals = create_overlay_signals(overlay_features, cohort_signals, base_models, meta_model, feature_columns)
print(f"Created overlay signals: {overlay_signals.shape}")

# Create Eligibility Matrix (Matching Original)
print("Creating eligibility matrix...")

def create_eligibility_matrix(signals_df, thresholds):
    """Create eligibility matrix based on signal thresholds (Matching Original)"""
    
    eligible_data = []
    
    for timeframe in signals_df['timeframe'].unique():
        tf_signals = signals_df[signals_df['timeframe'] == timeframe].copy()
        
        eligible_df = pd.DataFrame(False, index=tf_signals.index, columns=['pros', 'amateurs', 'mood', 'model'])
        
        # Eligibility rules (Matching Original)
        eligible_df['pros'] = tf_signals['S_top'].abs() >= thresholds['S_MIN']
        eligible_df['amateurs'] = tf_signals['S_bot'].abs() >= thresholds['S_MIN']
        eligible_df['mood'] = tf_signals['S_mood'].abs() >= thresholds['M_MIN']
        eligible_df['model'] = (tf_signals['confidence'] >= thresholds['CONF_MIN']) & (tf_signals['alpha'] >= thresholds['ALPHA_MIN'])
        
        eligible_df['timeframe'] = timeframe
        eligible_df['timestamp'] = tf_signals['timestamp']
        
        eligible_data.append(eligible_df)
    
    return pd.concat(eligible_data, ignore_index=True)

# Create eligibility matrix
thresholds = {
    'S_MIN': S_MIN,
    'M_MIN': M_MIN,
    'CONF_MIN': CONF_MIN,
    'ALPHA_MIN': ALPHA_MIN
}

eligibility_matrix = create_eligibility_matrix(overlay_signals, thresholds)
print(f"Created eligibility matrix: {eligibility_matrix.shape}")

# Create arm signals DataFrame (Matching Original Variable Names)
# For 5m timeframe (primary)
tf_5m_signals = overlay_signals[overlay_signals['timeframe'] == '5m'].copy()
tf_5m_eligible = eligibility_matrix[eligibility_matrix['timeframe'] == '5m'].copy()

# Align with backtest data
tf_5m_signals = tf_5m_signals.set_index('timestamp')
tf_5m_eligible = tf_5m_eligible.set_index('timestamp')

# Create arm signals matrix (Matching Original)
arm_signals_df = pd.DataFrame({
    'S_top': tf_5m_signals['S_top'],
    'S_bot': tf_5m_signals['S_bot'],
    'S_mood': tf_5m_signals['S_mood'],
    'S_model': tf_5m_signals['S_model'],
}, index=tf_5m_signals.index)

arm_eligible_df = pd.DataFrame(False, index=tf_5m_signals.index, columns=['pros','amateurs','mood','model'])
arm_eligible_df['pros'] = tf_5m_eligible['pros']
arm_eligible_df['amateurs'] = tf_5m_eligible['amateurs']
arm_eligible_df['mood'] = tf_5m_eligible['mood']
arm_eligible_df['model'] = tf_5m_eligible['model']

# Export to ndarray for backtest (Matching Original)
arm_signals = arm_signals_df[['S_top','S_bot','S_mood','S_model']].values
arm_eligible = arm_eligible_df[['pros','amateurs','mood','model']].values
arm_names = ['pros','amateurs','mood','model']

print(f"Arm signals shape: {arm_signals.shape}")
print(f"Arm eligible shape: {arm_eligible.shape}")
print(f"Arm names: {arm_names}")

# Display signal statistics
print(f"\n=== REAL SIGNAL STATISTICS ===")
for i, arm_name in enumerate(arm_names):
    signal_values = arm_signals[:, i]
    print(f"{arm_name}: mean={signal_values.mean():.4f}, std={signal_values.std():.4f}, range=[{signal_values.min():.4f}, {signal_values.max():.4f}]")

# Create ADV series (Matching Original)
adv20_by_ts = pd.Series(25000000.0, index=bt['timestamp'])  # $25M ADV
ANNUALIZER = np.sqrt(365*24*12)  # 5-minute bars

print("Signal generation complete!")


Creating overlay signals...
Creating signals for 5m...
Creating signals for 15m...
Creating signals for 1h...
Created overlay signals: (73440, 10)
Creating eligibility matrix...
Created eligibility matrix: (73440, 6)
Arm signals shape: (51840, 4)
Arm eligible shape: (51840, 4)
Arm names: ['pros', 'amateurs', 'mood', 'model']

=== REAL SIGNAL STATISTICS ===
pros: mean=0.0000, std=0.0000, range=[0.0000, 0.0000]
amateurs: mean=0.0000, std=0.0000, range=[0.0000, 0.0000]
mood: mean=0.0000, std=0.0000, range=[0.0000, 0.0000]
model: mean=0.0030, std=0.1266, range=[-0.9798, 0.9823]
Signal generation complete!


In [10]:
# Run Backtest (Matching Original)
print("Running backtest...")

# Prepare backtest data (Matching Original)
bt_df = bt.set_index('timestamp')
side_eps_vec = np.array([S_MIN, S_MIN, M_MIN, ALPHA_MIN], dtype=float)

# Run backtest (Matching Original)
Eq_df, Tr_df, Bu_df, met = run_allocator_backtest(
    bt_df=bt_df,
    arm_signals=arm_signals,
    arm_eligible=arm_eligible,
    adv_series=adv20_by_ts,
    cooldown_bars=COOLDOWN,
    cost_bps=COST_BP,
    impact_k=IMPACT_K,
    side_eps_vec=side_eps_vec,
    eps=1e-12,
)

print("Backtest Results:")
print(f"  Final Equity: {met['final_equity']:.4f}")
print(f"  Number of Trades: {met['n_trades']}")
print(f"  Sharpe Ratio: {met['sharpe']:.4f}")
print(f"  Sortino Ratio: {met['sortino']:.4f}")
print(f"  Max Drawdown: {met['maxDD']:.4f}")
print(f"  Turnover: {met['turnover']:.4f}")
print(f"  Hit Rate: {met['hit_rate']:.4f}")

# Display results
print(f"\nEquity DataFrame shape: {Eq_df.shape}")
print(f"Trades DataFrame shape: {Tr_df.shape}")
print(f"Bandit Updates DataFrame shape: {Bu_df.shape}")

# Show sample data
print(f"\nSample Equity Data:")
print(Eq_df.head())

print(f"\nSample Trades Data:")
print(Tr_df.head())

print(f"\nSample Bandit Updates:")
print(Bu_df.head())


Running backtest...
Backtest Results:
  Final Equity: nan
  Number of Trades: 1718
  Sharpe Ratio: nan
  Sortino Ratio: nan
  Max Drawdown: nan
  Turnover: 764.3017
  Hit Rate: nan

Equity DataFrame shape: (51840, 1)
Trades DataFrame shape: (1718, 4)
Bandit Updates DataFrame shape: (1021, 3)

Sample Equity Data:
                            equity
ts                                
1970-01-01 00:29:05.213400     NaN
1970-01-01 00:29:05.213700     NaN
1970-01-01 00:29:05.214000     NaN
1970-01-01 00:29:05.214300     NaN
1970-01-01 00:29:05.214600     NaN

Sample Trades Data:
                          ts  from_pos    to_pos  cost_bps
0 1970-01-01 00:29:05.234400  0.000000  0.791392       5.0
1 1970-01-01 00:29:05.244600  0.000000  0.461946       5.0
2 1970-01-01 00:29:05.244900  0.791392  0.000000       5.0
3 1970-01-01 00:29:05.245200  0.461946 -0.437476       5.0
4 1970-01-01 00:29:05.245500  0.000000 -0.409001       5.0

Sample Bandit Updates:
                          ts  chosen    re

In [11]:
# Extended grid with cooldown and confidence sweep (Matching Original)
import itertools
from datetime import datetime

COOLDOWN_GRID = [1, 3, 5, 10]
CONF_MIN_GRID_EXT = [0.6, 0.7, 0.8]
FEE_BPS_GRID = [3.0, 5.0, 10.0]
SIGMA_TARGET_GRID = [0.10, 0.20, 0.30]

GATE_VARIANTS = [
    None,
    {'name': 'cons2_adv0', 'consensus_min': 2, 'advantage_min': 0.0},
    {'name': 'cons3_adv0', 'consensus_min': 3, 'advantage_min': 0.0},
    {'name': 'cons2_adv0p02', 'consensus_min': 2, 'advantage_min': 0.02},
]

print("Grid search parameters defined:")
print(f"  Cooldown grid: {COOLDOWN_GRID}")
print(f"  Confidence grid: {CONF_MIN_GRID_EXT}")
print(f"  Fee grid: {FEE_BPS_GRID}")
print(f"  Sigma target grid: {SIGMA_TARGET_GRID}")
print(f"  Gate variants: {len(GATE_VARIANTS)}")


Grid search parameters defined:
  Cooldown grid: [1, 3, 5, 10]
  Confidence grid: [0.6, 0.7, 0.8]
  Fee grid: [3.0, 5.0, 10.0]
  Sigma target grid: [0.1, 0.2, 0.3]
  Gate variants: 4


In [12]:
# Run Extended Grid Search (Matching Original)
def run_extended_grid():
    """Run extended grid search optimization (Matching Original)"""
    rows = []
    stamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    
    print(f"Starting extended grid search at {stamp}")
    
    # Grid search over all combinations
    for cooldown in COOLDOWN_GRID:
        for conf_min in CONF_MIN_GRID_EXT:
            for fee_bps in FEE_BPS_GRID:
                for sigma_target in SIGMA_TARGET_GRID:
                    for gate_cfg in GATE_VARIANTS:
                        
                        # Update global parameters
                        global COOLDOWN, CONF_MIN, COST_BP, SIGMA_TARGET
                        COOLDOWN = cooldown
                        CONF_MIN = conf_min
                        COST_BP = fee_bps
                        SIGMA_TARGET = sigma_target
                        
                        # Run backtest with current configuration
                        try:
                            # Prepare side thresholds
                            side_eps_vec = np.array([S_MIN, S_MIN, M_MIN, ALPHA_MIN], dtype=float)
                            
                            # Run backtest
                            Eq, Tr, Bu, metrics = run_allocator_backtest(
                                bt_df=bt_df,
                                arm_signals=arm_signals,
                                arm_eligible=arm_eligible,
                                adv_series=adv20_by_ts,
                                cooldown_bars=COOLDOWN,
                                cost_bps=COST_BP,
                                impact_k=IMPACT_K,
                                side_eps_vec=side_eps_vec,
                                eps=1e-12,
                            )
                            
                            # Create row
                            row = {
                                'COOLDOWN': cooldown,
                                'CONF_MIN': conf_min,
                                'FEE_BPS': fee_bps,
                                'SIGMA_TARGET': sigma_target,
                                'GATE': gate_cfg['name'] if gate_cfg else 'none',
                                'final_equity': metrics['final_equity'],
                                'n_trades': metrics['n_trades'],
                                'sharpe': metrics['sharpe'],
                                'sortino': metrics['sortino'],
                                'maxDD': metrics['maxDD'],
                                'turnover': metrics['turnover'],
                                'hit_rate': metrics['hit_rate'],
                            }
                            
                            rows.append(row)
                            
                        except Exception as e:
                            print(f"Error in grid search: {e}")
                            continue
    
    # Create results DataFrame
    results_df = pd.DataFrame(rows)
    
    print(f"Grid search complete: {len(results_df)} configurations tested")
    
    # Find best configuration
    if not results_df.empty:
        best_idx = results_df['sharpe'].idxmax()
        best_config = results_df.loc[best_idx]
        
        print(f"\nBest Configuration:")
        print(f"  Sharpe: {best_config['sharpe']:.4f}")
        print(f"  Cooldown: {best_config['COOLDOWN']}")
        print(f"  Confidence: {best_config['CONF_MIN']}")
        print(f"  Fee BPS: {best_config['FEE_BPS']}")
        print(f"  Sigma Target: {best_config['SIGMA_TARGET']}")
        print(f"  Gate: {best_config['GATE']}")
    
    return results_df

# Run grid search
grid_results = run_extended_grid()

# Display top 10 results
print(f"\nTop 10 Configurations by Sharpe Ratio:")
top_10 = grid_results.nlargest(10, 'sharpe')
print(top_10[['COOLDOWN', 'CONF_MIN', 'FEE_BPS', 'SIGMA_TARGET', 'GATE', 'sharpe', 'final_equity', 'n_trades']])


Starting extended grid search at 20251025_090650


KeyboardInterrupt: 

In [ ]:
# Results Analysis & Visualization (Matching Original)
print("Creating visualizations...")

# Set up plotting style
plt.style.use('default')
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Unified Overlay Trading Results', fontsize=16)

# 1. Equity Curve
axes[0, 0].plot(Eq_df.index, Eq_df['equity'])
axes[0, 0].set_title('Equity Curve')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Equity')
axes[0, 0].grid(True)

# 2. Drawdown
drawdown = (Eq_df['equity'] / Eq_df['equity'].cummax() - 1.0) * 100
axes[0, 1].fill_between(Eq_df.index, drawdown, 0, alpha=0.3, color='red')
axes[0, 1].set_title('Drawdown (%)')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Drawdown %')
axes[0, 1].grid(True)

# 3. Bandit Arm Selection
if not Bu_df.empty:
    arm_counts = Bu_df['chosen'].value_counts().sort_index()
    arm_names = ['pros', 'amateurs', 'mood', 'model']
    arm_labels = [arm_names[i] if i < len(arm_names) else f'arm_{i}' for i in arm_counts.index]
    
    axes[1, 0].bar(arm_labels, arm_counts.values)
    axes[1, 0].set_title('Bandit Arm Selection Count')
    axes[1, 0].set_xlabel('Arm')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Performance Metrics
metrics_data = {
    'Metric': ['Sharpe', 'Sortino', 'Max DD', 'Hit Rate'],
    'Value': [met['sharpe'], met['sortino'], met['maxDD'], met['hit_rate']]
}
metrics_df = pd.DataFrame(metrics_data)

axes[1, 1].bar(metrics_df['Metric'], metrics_df['Value'])
axes[1, 1].set_title('Performance Metrics')
axes[1, 1].set_ylabel('Value')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Additional Analysis
print(f"\nDetailed Performance Analysis:")
print(f"  Total Return: {(met['final_equity'] - 1.0) * 100:.2f}%")
print(f"  Annualized Return: {((met['final_equity'] ** (252*24*12/len(Eq_df))) - 1) * 100:.2f}%")
print(f"  Volatility: {Eq_df['equity'].pct_change().std() * np.sqrt(252*24*12) * 100:.2f}%")
print(f"  Max Drawdown: {met['maxDD'] * 100:.2f}%")
print(f"  Calmar Ratio: {met['sharpe'] / met['maxDD'] if met['maxDD'] > 0 else np.nan:.4f}")

# Trade Analysis
if not Tr_df.empty:
    print(f"\nTrade Analysis:")
    print(f"  Total Trades: {len(Tr_df)}")
    print(f"  Average Trade Size: {Tr_df['to_pos'].abs().mean():.4f}")
    print(f"  Trade Frequency: {len(Tr_df) / (len(Eq_df) / (252*24*12)):.2f} trades/day")

# Bandit Analysis
if not Bu_df.empty:
    print(f"\nBandit Analysis:")
    for i, arm_name in enumerate(arm_names):
        arm_rewards = Bu_df[Bu_df['chosen'] == i]['reward']
        if len(arm_rewards) > 0:
            print(f"  {arm_name}: {len(arm_rewards)} selections, avg reward: {arm_rewards.mean():.6f}")

print("Visualization complete!")


In [ ]:
# Export Results and Models (Matching Original)
print("Exporting results and models...")

# Create models directory
os.makedirs('models', exist_ok=True)

# Export trained models
timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')

# Save base models
for name, model in base_models.items():
    model_path = f'models/{name}_model_{timestamp}.joblib'
    joblib.dump(model, model_path)
    print(f"Saved {name} model to {model_path}")

# Save meta model
meta_model_path = f'models/meta_model_{timestamp}.joblib'
joblib.dump(meta_model, meta_model_path)
print(f"Saved meta model to {meta_model_path}")

# Save feature columns
feature_columns_path = f'models/feature_columns_{timestamp}.json'
with open(feature_columns_path, 'w') as f:
    json.dump(feature_columns, f)
print(f"Saved feature columns to {feature_columns_path}")

# Create model manifest
manifest = {
    'timestamp': timestamp,
    'model_type': 'unified_overlay',
    'base_models': list(base_models.keys()),
    'feature_columns': feature_columns,
    'overlay_timeframes': OVERLAY_TIMEFRAMES,
    'rollup_windows': ROLLUP_WINDOWS,
    'timeframe_weights': TIMEFRAME_WEIGHTS,
    'training_params': {
        'S_MIN': S_MIN,
        'M_MIN': M_MIN,
        'CONF_MIN': CONF_MIN,
        'ALPHA_MIN': ALPHA_MIN,
        'COOLDOWN': COOLDOWN,
        'COST_BP': COST_BP,
        'SIGMA_TARGET': SIGMA_TARGET,
    },
    'performance_metrics': met,
    'grid_search_results': grid_results.to_dict('records') if not grid_results.empty else []
}

manifest_path = f'models/manifest_{timestamp}.json'
with open(manifest_path, 'w') as f:
    json.dump(manifest, f, indent=2)
print(f"Saved model manifest to {manifest_path}")

# Export backtest results
results_dir = f'results_{timestamp}'
os.makedirs(results_dir, exist_ok=True)

# Save equity curve
Eq_df.to_csv(f'{results_dir}/equity_curve.csv')
print(f"Saved equity curve to {results_dir}/equity_curve.csv")

# Save trades
Tr_df.to_csv(f'{results_dir}/trades.csv', index=False)
print(f"Saved trades to {results_dir}/trades.csv")

# Save bandit updates
Bu_df.to_csv(f'{results_dir}/bandit_updates.csv', index=False)
print(f"Saved bandit updates to {results_dir}/bandit_updates.csv")

# Save grid search results
grid_results.to_csv(f'{results_dir}/grid_search_results.csv', index=False)
print(f"Saved grid search results to {results_dir}/grid_search_results.csv")

# Save overlay signals
overlay_signals.to_csv(f'{results_dir}/overlay_signals.csv', index=False)
print(f"Saved overlay signals to {results_dir}/overlay_signals.csv")

# Save eligibility matrix
eligibility_matrix.to_csv(f'{results_dir}/eligibility_matrix.csv', index=False)
print(f"Saved eligibility matrix to {results_dir}/eligibility_matrix.csv")

# Create summary report
summary_report = f"""
# Unified Overlay Trading Results Summary

## Model Training
- **Timestamp**: {timestamp}
- **Model Type**: Unified Overlay (Single model for all timeframes)
- **Base Models**: {', '.join(base_models.keys())}
- **Feature Columns**: {len(feature_columns)}
- **Overlay Timeframes**: {', '.join(OVERLAY_TIMEFRAMES)}

## Performance Metrics
- **Final Equity**: {met['final_equity']:.4f}
- **Sharpe Ratio**: {met['sharpe']:.4f}
- **Sortino Ratio**: {met['sortino']:.4f}
- **Max Drawdown**: {met['maxDD']:.4f}
- **Number of Trades**: {met['n_trades']}
- **Hit Rate**: {met['hit_rate']:.4f}

## Grid Search Results
- **Total Configurations Tested**: {len(grid_results)}
- **Best Sharpe Ratio**: {grid_results['sharpe'].max():.4f} if not grid_results.empty else 'N/A'

## Files Exported
- Models: models/
- Results: {results_dir}/
- Manifest: {manifest_path}

## Next Steps
1. Copy models to MetaStackerBandit/live_demo/models/
2. Update MetaStackerBandit configuration
3. Deploy overlay system
4. Monitor performance
"""

with open(f'{results_dir}/summary_report.md', 'w') as f:
    f.write(summary_report)

print(f"Saved summary report to {results_dir}/summary_report.md")

print("\nExport complete!")
print(f"All files saved with timestamp: {timestamp}")
print(f"Results directory: {results_dir}")
print(f"Models directory: models/")

# Display final summary
print(f"\n=== FINAL SUMMARY ===")
print(f"Unified Overlay Architecture Implementation Complete!")
print(f"Single model trained on 5m data for all timeframes")
print(f"Performance: Sharpe={met['sharpe']:.4f}, MaxDD={met['maxDD']:.4f}")
print(f"Ready for deployment to MetaStackerBandit")


# ML Trading Pipeline - Unified Overlay Implementation

## Overview
This notebook implements the unified overlay architecture for MetaStackerBandit with **one-to-one parity** to the original Bandit New notebook. Instead of training separate models for 5m, 1h, and 12h timeframes, we train a single model on 5-minute data that can generate signals for all timeframes using rollup overlays.

## Key Changes from Original:
1. **Single Model Training**: One model trained on 5m data for all timeframes
2. **Overlay Feature Engineering**: Rollup features for 15m and 1h from 5m base
3. **Unified Feature Schema**: Same features across all timeframes
4. **Multi-Timeframe Signals**: Generate signals for 5m, 15m, 1h simultaneously
5. **Enhanced Bandit**: Multi-level bandit for timeframe and signal selection

## Structure (Matching Original):
1. Import Required Libraries
2. Configuration & Parameters
3. Data Loading and Processing
4. Feature Engineering (Unified)
5. Trading Signal Generation (Multi-Arm Bandit)
6. Backtesting Engine
7. Grid Search Optimization
8. Results Analysis & Visualization
9. Export & Deployment


## 1. Import Required Libraries


In [ ]:
# Imports (Matching Original Bandit New)
import pandas as pd, numpy as np, warnings, os, json, joblib
from datetime import datetime, timedelta
from collections import defaultdict, deque
import scipy.stats as stats
from scipy.optimize import minimize_scalar
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier, HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNetCV, HuberRegressor, LogisticRegression
from sklearn.model_selection import KFold, TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score, classification_report, confusion_matrix, log_loss
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.base import clone
from typing import Dict, List, Tuple, Optional, Union
import random
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

# Deterministic seeds
np.random.seed(42)
random.seed(42)


## 2. Configuration & Parameters


In [ ]:
# Single-source threshold & cost config for allocator-only mode (Matching Original)
# These globals are consumed by consolidated arms and backtest cells.
try:
    S_MIN
except NameError:
    S_MIN = 0.12
try:
    M_MIN
except NameError:
    M_MIN = 0.12
try:
    CONF_MIN
except NameError:
    CONF_MIN = 0.60
try:
    ALPHA_MIN
except NameError:
    ALPHA_MIN = 0.10
try:
    COOLDOWN
except NameError:
    COOLDOWN = 1
try:
    COST_BP
except NameError:
    COST_BP = 5.0
try:
    IMPACT_K
except NameError:
    IMPACT_K = 0.0

# Risk and execution controls (Matching Original)
try:
    SIGMA_TARGET
except NameError:
    SIGMA_TARGET = 0.20  # per-bar target scaler proxy
try:
    POS_MAX
except NameError:
    POS_MAX = 1.0
try:
    DD_STOP
except NameError:
    DD_STOP = 0.05
try:
    LATENCY_BARS
except NameError:
    LATENCY_BARS = 0
try:
    SLIPPAGE_BPS
except NameError:
    SLIPPAGE_BPS = 0.0
try:
    COST_CONVENTION
except NameError:
    COST_CONVENTION = 'per_transition'  # or 'per_roundtrip'
try:
    SMOOTH_BETA
except NameError:
    SMOOTH_BETA = 0.0

# Overlay-specific parameters (NEW)
OVERLAY_TIMEFRAMES = ["5m", "15m", "1h"]
ROLLUP_WINDOWS = {"15m": 3, "1h": 12}  # 3x5m=15m, 12x5m=1h
TIMEFRAME_WEIGHTS = {"5m": 0.5, "15m": 0.3, "1h": 0.2}

# Feature engineering parameters (unified across timeframes)
MOMENTUM_PERIODS = [1, 3, 6]  # Consistent momentum periods
EMA_PERIOD = 20  # Consistent EMA period
ROLLING_WINDOW = 100  # Consistent rolling window
RV_WINDOWS = {"1h": 12, "15m": 3, "1d": 288}  # RV windows

print("Configuration loaded:")
print(f"  Signal thresholds: S_MIN={S_MIN}, M_MIN={M_MIN}, CONF_MIN={CONF_MIN}, ALPHA_MIN={ALPHA_MIN}")
print(f"  Risk controls: SIGMA_TARGET={SIGMA_TARGET}, POS_MAX={POS_MAX}, DD_STOP={DD_STOP}")
print(f"  Overlay timeframes: {OVERLAY_TIMEFRAMES}")
print(f"  Rollup windows: {ROLLUP_WINDOWS}")
print(f"  Timeframe weights: {TIMEFRAME_WEIGHTS}")


## 3. Data Loading and Processing


In [ ]:
# Load data (Matching Original Structure)
print("Loading data...")

# Load OHLCV data
df = pd.read_csv('ohlc_btc_5m.csv') if os.path.exists('ohlc_btc_5m.csv') else pd.DataFrame()
if not df.empty:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    print(f"Loaded {len(df)} 5m bars")
else:
    print("Error: No OHLCV data found")

# Load funding data
funding_df = pd.read_csv('funding_btc.csv') if os.path.exists('funding_btc.csv') else pd.DataFrame()
if not funding_df.empty:
    funding_df['timestamp'] = pd.to_datetime(funding_df['timestamp'])
    funding_df = funding_df.sort_values('timestamp').reset_index(drop=True)
    print(f"Loaded {len(funding_df)} funding records")

# Load cohort data
cohort_top = pd.read_csv('top_cohort.csv') if os.path.exists('top_cohort.csv') else pd.DataFrame()
cohort_bot = pd.read_csv('bottom_cohort.csv') if os.path.exists('bottom_cohort.csv') else pd.DataFrame()

# Process cohort addresses (Matching Original)
for cdf in (cohort_top, cohort_bot):
    if not cdf.empty and 'user' not in cdf.columns:
        col = next((c for c in ['Account', 'address', 'user', 'addr'] if c in cdf.columns), None)
        if col: cdf['user'] = cdf[col].astype(str)

cohort_top_users = set(cohort_top.get('user', pd.Series(dtype=str)).dropna().astype(str))
cohort_bot_users = set(cohort_bot.get('user', pd.Series(dtype=str)).dropna().astype(str))
cohort_addresses = cohort_top_users | cohort_bot_users

print(f"Cohort addresses: {len(cohort_addresses)}")

# Load fills data (Matching Original Processing)
fills = pd.read_csv('historical_trades_btc.csv', low_memory=False) if os.path.exists('historical_trades_btc.csv') else pd.DataFrame()
if not fills.empty:
    if 'user' not in fills.columns:
        if 'Account' in fills.columns: fills['user'] = fills['Account'].astype(str)
    
    # Parse timestamps (Matching Original)
    ts_col = 'Timestamp' if 'Timestamp' in fills.columns else (next((c for c in ['timestamp', 'ts'] if c in fills.columns), None))
    ts_raw = pd.to_numeric(fills[ts_col], errors='coerce') if ts_col else pd.Series(dtype='float64')
    if ts_raw.notna().any():
        med = ts_raw.dropna().median()
        unit = 'ns' if med>1e14 else ('ms' if med>1e12 else 's')
        fills['timestamp'] = pd.to_datetime(ts_raw, unit=unit, errors='coerce')
    else:
        time_col = next((c for c in ['Timestamp IST', 'time'] if c in fills.columns), None)
        fills['timestamp'] = pd.to_datetime(fills[time_col], errors='coerce') if time_col else pd.NaT
    
    # Map helpers (Matching Original)
    if 'Execution Price' in fills.columns and 'px' not in fills.columns:
        fills['px'] = pd.to_numeric(fills['Execution Price'], errors='coerce')
    if 'Size Tokens' in fills.columns and 'sz' not in fills.columns:
        fills['sz'] = pd.to_numeric(fills['Size Tokens'], errors='coerce')
    if 'Size USD' in fills.columns and 'notional' not in fills.columns:
        fills['notional'] = pd.to_numeric(fills['Size USD'], errors='coerce')
    if 'Side' in fills.columns and 'side' not in fills.columns:
        fills['side'] = fills['Side'].map({'BUY':'B','SELL':'A'}).fillna(fills['Side'].astype(str))
    
    print(f"Loaded {len(fills)} fills")

print("Data loading complete!")


## 4. Unified Feature Engineering


In [ ]:
# Unified Feature Engineering (Matching Original + Overlay)
print("Creating unified features...")

# Create base features from 5m data
features_df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']].copy()

# Price and returns (Matching Original)
features_df['price'] = features_df['close']
features_df['returns'] = features_df['price'].pct_change()

# Technical indicators (unified parameters)
features_df['hl_range'] = features_df['high'] - features_df['low']
features_df['oc_range'] = np.abs(features_df['open'] - features_df['close'])
features_df['typical_price'] = (features_df['high'] + features_df['low'] + features_df['close']) / 3
features_df['true_range'] = np.maximum(
    features_df['high'] - features_df['low'],
    np.maximum(
        np.abs(features_df['high'] - features_df['close'].shift(1)),
        np.abs(features_df['low'] - features_df['close'].shift(1))
    )
)

# Momentum features (consistent periods)
for h in MOMENTUM_PERIODS:
    features_df[f'mom_{h}'] = features_df['price'].pct_change(periods=h)

# EMA and mean reversion (unified parameters)
features_df['ema20'] = features_df['price'].ewm(span=EMA_PERIOD, adjust=False).mean()
features_df['mr_ema20'] = (features_df['price'] - features_df['ema20']) / features_df['ema20']

# Z-score normalization (unified rolling window)
rolling_mean = features_df['price'].rolling(window=ROLLING_WINDOW, min_periods=20).mean()
rolling_std = features_df['price'].rolling(window=ROLLING_WINDOW, min_periods=20).std()
features_df['mr_ema20_z'] = (features_df['price'] - rolling_mean) / (rolling_std + 1e-8)

# Volatility features (unified RV windows)
features_df['rv_1h'] = features_df['returns'].rolling(window=RV_WINDOWS['1h'], min_periods=6).apply(lambda x: (x**2).sum())
features_df['rv_15m'] = features_df['returns'].rolling(window=RV_WINDOWS['15m'], min_periods=2).apply(lambda x: (x**2).sum())
features_df['rv_1d'] = features_df['returns'].rolling(window=RV_WINDOWS['1d'], min_periods=50).apply(lambda x: (x**2).sum())

# Volatility regime
rv_threshold = features_df['rv_1h'].rolling(window=100, min_periods=25).quantile(0.75)
features_df['regime_high_vol'] = (features_df['rv_1h'] > rv_threshold).astype(int)

# Enhanced volatility features (Matching Original)
if {'high','low','close','open'}.issubset(features_df.columns):
    gk_vol = np.log(features_df['high']/features_df['low'])**2 / 2 - (2*np.log(2)-1) * np.log(features_df['close']/features_df['open'])**2
    features_df['gk_volatility'] = gk_vol.rolling(12, min_periods=6).mean()
    
    parkinson_vol = np.log(features_df['high']/features_df['low'])**2 / (4 * np.log(2))
    features_df['parkinson_volatility'] = parkinson_vol.rolling(12, min_periods=6).mean()
    
    vol_percentile_20 = features_df['gk_volatility'].rolling(100, min_periods=50).quantile(0.2)
    vol_percentile_80 = features_df['gk_volatility'].rolling(100, min_periods=50).quantile(0.8)
    features_df['vol_regime_low'] = (features_df['gk_volatility'] <= vol_percentile_20).astype('float')
    features_df['vol_regime_high'] = (features_df['gk_volatility'] >= vol_percentile_80).astype('float')

# Jump detection (Matching Original)
return_std = features_df['returns'].rolling(50, min_periods=25).std()
features_df['jump_indicator'] = (np.abs(features_df['returns']) > 3 * return_std).astype('float')
features_df['jump_magnitude'] = np.abs(features_df['returns']) / (return_std + 1e-8)

# Volume features (Matching Original)
features_df['volume_intensity'] = features_df['volume'] / features_df['volume'].rolling(50, min_periods=25).mean()
features_df['price_volume_corr'] = features_df['returns'].rolling(20, min_periods=10).corr(features_df['volume'].pct_change())

# Price efficiency (Matching Original)
features_df['price_efficiency'] = np.abs(features_df['close'] - features_df['open']) / (features_df['high'] - features_df['low'] + 1e-8)

# VWAP momentum (Matching Original)
features_df['vwap'] = (features_df['high'] + features_df['low'] + features_df['close']) / 3
features_df['vwap_momentum'] = features_df['vwap'].pct_change(periods=5)

# Depth proxy (simplified)
features_df['depth_proxy'] = features_df['volume'] / (features_df['hl_range'] + 1e-8)

# Funding features (if available)
if funding_df is not None and not funding_df.empty:
    funding_df['timestamp'] = pd.to_datetime(funding_df['timestamp'])
    features_df = features_df.merge(funding_df[['timestamp', 'funding_rate']], on='timestamp', how='left')
    features_df['funding_rate'] = features_df['funding_rate'].fillna(0.0)
    features_df['funding_momentum_1h'] = features_df['funding_rate'].rolling(12, min_periods=6).mean()
else:
    features_df['funding_rate'] = 0.0
    features_df['funding_momentum_1h'] = 0.0

# Flow features (placeholder - would need actual flow data)
features_df['flow_diff'] = 0.0

print(f"Created unified features: {features_df.shape}")
print(f"Feature columns: {list(features_df.columns)}")


In [ ]:
# Create Overlay Features (NEW - Overlay Architecture)
print("Creating overlay features...")

def create_overlay_features(base_features_df, timeframe, rollup_window):
    """Create overlay features by rolling up base 5m features"""
    if timeframe == "5m":
        return base_features_df.copy()
    
    # Create rollup bars
    rollup_df = base_features_df.copy()
    rollup_df['rollup_group'] = rollup_df.index // rollup_window
    
    # Get all available columns for aggregation
    agg_dict = {
        'timestamp': 'last',
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'price': 'last',
        'returns': lambda x: (x.iloc[-1] / x.iloc[0] - 1) if len(x) > 1 else 0,
        'mom_1': 'last',
        'mom_3': 'last', 
        'mom_6': 'last',
        'mr_ema20_z': 'last',
        'rv_1h': 'mean',
        'regime_high_vol': 'mean',
        'gk_volatility': 'mean',
        'jump_magnitude': 'mean',
        'volume_intensity': 'mean',
        'price_efficiency': 'mean',
        'vwap_momentum': 'last',
        'depth_proxy': 'mean',
        'funding_rate': 'mean',
        'funding_momentum_1h': 'mean',
        'flow_diff': 'sum'
    }
    
    # Add price_volume_corr if it exists
    if 'price_volume_corr' in rollup_df.columns:
        agg_dict['price_volume_corr'] = 'mean'
    
    overlay_features = rollup_df.groupby('rollup_group').agg(agg_dict).reset_index(drop=True)
    
    # Recalculate some features for the new timeframe
    overlay_features['price'] = overlay_features['close']
    overlay_features['hl_range'] = overlay_features['high'] - overlay_features['low']
    overlay_features['typical_price'] = (overlay_features['high'] + overlay_features['low'] + overlay_features['close']) / 3
    
    return overlay_features

# Create overlay features for all timeframes
overlay_features = {}
overlay_features['5m'] = create_overlay_features(features_df, '5m', 1)
overlay_features['15m'] = create_overlay_features(features_df, '15m', ROLLUP_WINDOWS['15m'])
overlay_features['1h'] = create_overlay_features(features_df, '1h', ROLLUP_WINDOWS['1h'])

print(f"Overlay features created:")
for tf, features in overlay_features.items():
    print(f"  {tf}: {features.shape[0]} bars")

# Store base features for backtesting
bt = features_df.copy()  # Matching original variable name
bt['returns'] = bt['price'].pct_change()  # Ensure returns column exists


## 5. Trading Signal Generation

### Advanced Signal Pipeline with Multi-Arm Bandit Strategy Selection


In [ ]:
# Bandit allocator: class and backtest function (Matching Original)
import numpy as np
import pandas as pd
from typing import Tuple, Optional

class SimpleThompsonBandit:
    def __init__(self, n_arms: int):
        self.counts = np.zeros(n_arms, dtype=float)
        self.means = np.zeros(n_arms, dtype=float)
        self.vars = np.ones(n_arms, dtype=float)
        
    def select(self, eligible: np.ndarray) -> int:
        """Select arm using Thompson Sampling"""
        if not np.any(eligible):
            return 0
        
        # Sample from posterior
        samples = np.random.normal(self.means, np.sqrt(self.vars))
        
        # Only consider eligible arms
        samples[~eligible] = -np.inf
        
        return int(np.argmax(samples))
    
    def update(self, arm: int, reward: float):
        """Update arm statistics with new reward"""
        self.counts[arm] += 1
        n = self.counts[arm]
        
        # Update mean
        self.means[arm] = (self.means[arm] * (n - 1) + reward) / n
        
        # Update variance (simplified)
        if n > 1:
            self.vars[arm] = 1.0 / n

print("Bandit class defined")


In [ ]:
# Comprehensive Backtesting Engine (Matching Original)
def run_allocator_backtest(
    bt_df: pd.DataFrame,
    arm_signals: np.ndarray,
    arm_eligible: np.ndarray,
    adv_series: pd.Series,
    cooldown_bars: int,
    cost_bps: float,
    impact_k: float,
    side_eps_vec: np.ndarray,
    eps: float = 1e-12,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, Dict]:
    """Comprehensive backtesting with bandit allocation (Matching Original)"""
    
    n = len(bt_df)
    rets = bt_df['returns'].values if 'returns' in bt_df.columns else np.zeros(n)
    
    # ADV handling (Matching Original)
    adv_valid = isinstance(adv_series, pd.Series) and not adv_series.dropna().empty
    adv_arr = adv_series.reindex(bt_df.index).to_numpy() if adv_valid else np.ones(n, dtype=float)
    impact_k_eff = impact_k if adv_valid else 0.0
    
    bandit = SimpleThompsonBandit(n_arms=arm_signals.shape[1])
    
    pos = 0.0
    pos_smooth = 0.0
    last_flip_idx = -10**9
    
    exec_pos_buffer = deque(maxlen=cooldown_bars + 1)
    exec_pos_buffer.append(0.0)
    
    records_eq = []
    records_tr = []
    records_bu = []
    
    cum_equity = 1.0
    equity_series = np.ones(n, dtype=float)
    
    sigma_target = float(globals().get('SIGMA_TARGET', 0.20))
    pos_max = float(globals().get('POS_MAX', 1.0))
    dd_stop = float(globals().get('DD_STOP', 0.05))
    latency_k = int(globals().get('LATENCY_BARS', 0))
    
    for t in range(n):
        exec_pos = exec_pos_buffer[0] if latency_k > 0 else pos_smooth
        
        # Check eligibility and select arm (Matching Original)
        elig = arm_eligible[t] if t < len(arm_eligible) else np.array([False]*arm_signals.shape[1])
        desired_side = pos
        chosen = None
        
        if np.any(elig):
            chosen = bandit.select(elig)
            raw_val = float(arm_signals[t, chosen])
            th = float(side_eps_vec[chosen]) if (side_eps_vec is not None) else 0.0
            
            if abs(raw_val) < th:
                desired_side = 0.0
            else:
                desired_side = np.sign(raw_val) * pos_max
        
        # Position sizing with volatility targeting (Matching Original)
        if t > 0 and abs(rets[t]) > eps:
            vol_est = np.std(rets[max(0, t-20):t+1])
            if vol_est > eps:
                vol_scaler = sigma_target / (vol_est * np.sqrt(252 * 24 * 12))  # Annualized
                desired_side *= min(vol_scaler, 2.0)  # Cap at 2x
        
        # Cooldown logic (Matching Original)
        if abs(desired_side - exec_pos) > eps and (t - last_flip_idx) >= cooldown_bars:
            exec_pos_buffer.append(desired_side)
            last_exec_pos = exec_pos
            
            # Cost calculation (Matching Original)
            cost_bps_eff = cost_bps
            if impact_k_eff > 0 and adv_valid:
                notional_change = abs(desired_side - exec_pos)
                impact_bps = impact_k_eff * notional_change / (adv_arr[t] + eps)
                cost_bps_eff += impact_bps
            
            records_tr.append((bt_df.index[t], exec_pos, desired_side, cost_bps_eff))
            last_exec_pos = exec_pos
            last_flip_idx = t
        
        pnl = rets[t] * (exec_pos_buffer[0] if latency_k > 0 else exec_pos)
        pnl -= (cost_bps / 10000.0) if cost_bps > 0 else 0.0
        cum_equity *= (1.0 + pnl)
        equity_series[t] = cum_equity
        records_eq.append((bt_df.index[t], cum_equity))
        
        if chosen is not None:
            bandit.update(chosen, pnl)
            records_bu.append((bt_df.index[t], int(chosen), float(pnl)))
        
        # Drawdown stop (Matching Original)
        if dd_stop > 0:
            peak = np.max(equity_series[:t+1])
            if peak > 0 and (cum_equity / peak - 1.0) < -dd_stop:
                exec_pos_buffer.append(0.0)
                last_exec_pos = 0.0
        
        pos_smooth = exec_pos_buffer[0] if exec_pos_buffer else 0.0
    
    Eq = pd.DataFrame.from_records(records_eq, columns=['ts', 'equity']).set_index('ts')
    Tr = pd.DataFrame.from_records(records_tr, columns=['ts', 'from_pos', 'to_pos', 'cost_bps'])
    Bu = pd.DataFrame.from_records(records_bu, columns=['ts', 'chosen', 'reward'])
    
    # Calculate performance metrics (Matching Original)
    eq_rets = Eq['equity'].pct_change().dropna()
    annualizer = float(globals().get('ANNUALIZER', np.sqrt(365*24*12)))
    
    sharpe = float(annualizer * eq_rets.mean() / (eq_rets.std() if eq_rets.std() != 0 else np.nan)) if len(eq_rets) else np.nan
    
    # Sortino ratio (Matching Original)
    downside_rets = eq_rets[eq_rets < 0]
    sortino = float(annualizer * eq_rets.mean() / (downside_rets.std() if len(downside_rets) > 0 else np.nan)) if len(eq_rets) else np.nan
    
    maxdd = float(-(Eq['equity'] / Eq['equity'].cummax() - 1.0).min()) if not Eq.empty else np.nan
    
    # Turnover (Matching Original)
    turnover = float(np.abs(Tr['to_pos'].astype(float) - Tr['from_pos'].astype(float)).sum()) if not Tr.empty else 0.0
    
    # Hit rate (Matching Original)
    hit_rate = float((Tr['pnl_$'] > 0).sum() / max(len(Tr), 1)) if not Tr.empty and 'pnl_$' in Tr.columns else np.nan
    
    metrics = {
        'final_equity': float(Eq['equity'].iloc[-1]) if len(Eq) else 1.0,
        'n_trades': int(len(Tr)),
        'sharpe': sharpe,
        'sortino': sortino,
        'maxDD': maxdd,
        'turnover': turnover,
        'hit_rate': hit_rate,
    }
    
    return Eq, Tr, Bu, metrics

print("Backtesting engine defined")


In [ ]:
# Model Training and Signal Generation (Matching Original + Overlay)
print("Training unified model...")

# Prepare training data for 5m (base timeframe)
def prepare_training_data(features_df, cohort_signals_df, target_horizon=1):
    """Prepare training data for the unified model (Matching Original)"""
    # Merge features with cohort signals
    training_df = features_df.merge(cohort_signals_df, on='timestamp', how='left')
    
    # Fill missing cohort signals
    training_df['S_top'] = training_df['S_top'].fillna(0.0)
    training_df['S_bot'] = training_df['S_bot'].fillna(0.0)
    
    # Create target variable (future returns)
    training_df['future_return'] = training_df['price'].pct_change(periods=target_horizon).shift(-target_horizon)
    
    # Create classification target (3-class: down, neutral, up)
    training_df['target'] = 1  # neutral
    training_df.loc[training_df['future_return'] > 0.001, 'target'] = 2  # up
    training_df.loc[training_df['future_return'] < -0.001, 'target'] = 0  # down
    
    # Select features for training (Matching Original)
    feature_columns = [
        'mom_1', 'mom_3', 'mom_6', 'mr_ema20_z', 'rv_1h', 'regime_high_vol',
        'gk_volatility', 'jump_magnitude', 'volume_intensity', 'price_efficiency',
        'price_volume_corr', 'vwap_momentum', 'depth_proxy', 'funding_rate',
        'funding_momentum_1h', 'flow_diff', 'S_top', 'S_bot'
    ]
    
    # Filter available features
    available_features = [col for col in feature_columns if col in training_df.columns]
    
    # Prepare training data
    X = training_df[available_features].fillna(0.0)
    y = training_df['target']
    
    # Remove rows with missing targets
    valid_mask = ~y.isna()
    X = X[valid_mask]
    y = y[valid_mask]
    
    return X, y, available_features

# Process cohort signals (Matching Original)
def process_cohort_signals(fills_df, cohort_top_users, cohort_bot_users):
    """Process cohort signals from fills data (Matching Original)"""
    if fills_df.empty:
        return pd.DataFrame()
    
    # Filter cohort fills
    cohort_fills = fills_df[fills_df['user'].isin(cohort_top_users | cohort_bot_users)].copy()
    
    if cohort_fills.empty:
        return pd.DataFrame()
    
    # Calculate signals (Matching Original)
    cohort_fills['side_numeric'] = cohort_fills['side'].map({'B': 1, 'A': -1, 'BUY': 1, 'SELL': -1}).fillna(0)
    cohort_fills['impact'] = cohort_fills['side_numeric'] * cohort_fills['notional']
    
    # Group by time windows (Matching Original)
    cohort_fills['time_window'] = pd.to_datetime(cohort_fills['timestamp']).dt.floor('5T')
    
    signals_df = cohort_fills.groupby('time_window').agg({
        'impact': 'sum',
        'notional': 'sum',
        'user': 'count'
    }).reset_index()
    
    signals_df.columns = ['timestamp', 'net_impact', 'total_notional', 'trade_count']
    
    # Calculate normalized signals (Matching Original)
    signals_df['S_top'] = signals_df['net_impact'] / (signals_df['total_notional'] + 1e-8)
    signals_df['S_bot'] = signals_df['net_impact'] / (signals_df['total_notional'] + 1e-8)
    
    return signals_df

# Process cohort signals
cohort_signals = process_cohort_signals(fills, cohort_top_users, cohort_bot_users)
print(f"Processed cohort signals: {cohort_signals.shape}")

# Train unified model on 5m data
X_train, y_train, feature_columns = prepare_training_data(overlay_features['5m'], cohort_signals)

print(f"Training data shape: {X_train.shape}")
print(f"Target distribution: {y_train.value_counts().to_dict()}")
print(f"Feature columns: {len(feature_columns)}")

# Train base models (Matching Original)
base_models = {
    'xgb': HistGradientBoostingClassifier(max_iter=100, learning_rate=0.1, max_depth=6, random_state=42),
    'hgb': HistGradientBoostingClassifier(max_iter=150, learning_rate=0.05, max_depth=8, random_state=42),
    'lasso': LogisticRegression(C=0.1, penalty='l1', solver='liblinear', random_state=42),
    'logit': LogisticRegression(C=1.0, penalty='l2', random_state=42)
}

# Train models
for name, model in base_models.items():
    model.fit(X_train, y_train)
    print(f"Trained {name} model")

# Create meta-model (Matching Original)
base_predictions = {}
for name, model in base_models.items():
    pred = model.predict_proba(X_train)
    base_predictions[name] = pred

meta_features = np.hstack([base_predictions[name] for name in base_models.keys()])
meta_model = LogisticRegression(C=1.0, random_state=42)
meta_model.fit(meta_features, y_train)

print(f"Model training complete: {len(feature_columns)} features")


In [ ]:
# Create Overlay Signals (NEW - Multi-Timeframe Signal Generation)
print("Creating overlay signals...")

def create_overlay_signals(unified_features, cohort_signals, base_models, meta_model, feature_columns):
    """Create multi-arm signals for overlay timeframes (Matching Original Structure)"""
    
    signals_data = []
    
    for timeframe, features_df in unified_features.items():
        print(f"Creating signals for {timeframe}...")
        
        # Merge with cohort signals
        merged_df = features_df.merge(cohort_signals, on='timestamp', how='left')
        merged_df['S_top'] = merged_df['S_top'].fillna(0.0)
        merged_df['S_bot'] = merged_df['S_bot'].fillna(0.0)
        
        # Prepare features
        X = merged_df[feature_columns].fillna(0.0)
        
        # Get model predictions (Matching Original)
        base_preds = {}
        for name, model in base_models.items():
            pred = model.predict_proba(X)
            base_preds[name] = pred
        
        meta_features = np.hstack([base_preds[name] for name in base_models.keys()])
        meta_pred = meta_model.predict_proba(meta_features)
        
        # Extract model signal (Matching Original)
        p_up = meta_pred[:, 2]  # up probability
        p_down = meta_pred[:, 0]  # down probability
        s_model = p_up - p_down
        
        # Create signals for this timeframe (Matching Original Structure)
        timeframe_signals = pd.DataFrame({
            'timestamp': merged_df['timestamp'],
            'timeframe': timeframe,
            'S_top': merged_df['S_top'],
            'S_bot': merged_df['S_bot'],
            'S_mood': merged_df['S_top'] + merged_df['S_bot'],  # Combined mood signal
            'S_model': s_model,
            'p_up': p_up,
            'p_down': p_down,
            'confidence': np.maximum(p_up, p_down),
            'alpha': np.abs(s_model)
        })
        
        signals_data.append(timeframe_signals)
    
    return pd.concat(signals_data, ignore_index=True)

# Create overlay signals
overlay_signals = create_overlay_signals(overlay_features, cohort_signals, base_models, meta_model, feature_columns)
print(f"Created overlay signals: {overlay_signals.shape}")

# Create Eligibility Matrix (Matching Original)
print("Creating eligibility matrix...")

def create_eligibility_matrix(signals_df, thresholds):
    """Create eligibility matrix based on signal thresholds (Matching Original)"""
    
    eligible_data = []
    
    for timeframe in signals_df['timeframe'].unique():
        tf_signals = signals_df[signals_df['timeframe'] == timeframe].copy()
        
        eligible_df = pd.DataFrame(False, index=tf_signals.index, columns=['pros', 'amateurs', 'mood', 'model'])
        
        # Eligibility rules (Matching Original)
        eligible_df['pros'] = tf_signals['S_top'].abs() >= thresholds['S_MIN']
        eligible_df['amateurs'] = tf_signals['S_bot'].abs() >= thresholds['S_MIN']
        eligible_df['mood'] = tf_signals['S_mood'].abs() >= thresholds['M_MIN']
        eligible_df['model'] = (tf_signals['confidence'] >= thresholds['CONF_MIN']) & (tf_signals['alpha'] >= thresholds['ALPHA_MIN'])
        
        eligible_df['timeframe'] = timeframe
        eligible_df['timestamp'] = tf_signals['timestamp']
        
        eligible_data.append(eligible_df)
    
    return pd.concat(eligible_data, ignore_index=True)

# Create eligibility matrix
thresholds = {
    'S_MIN': S_MIN,
    'M_MIN': M_MIN,
    'CONF_MIN': CONF_MIN,
    'ALPHA_MIN': ALPHA_MIN
}

eligibility_matrix = create_eligibility_matrix(overlay_signals, thresholds)
print(f"Created eligibility matrix: {eligibility_matrix.shape}")

# Create arm signals DataFrame (Matching Original Variable Names)
# For 5m timeframe (primary)
tf_5m_signals = overlay_signals[overlay_signals['timeframe'] == '5m'].copy()
tf_5m_eligible = eligibility_matrix[eligibility_matrix['timeframe'] == '5m'].copy()

# Align with backtest data
tf_5m_signals = tf_5m_signals.set_index('timestamp')
tf_5m_eligible = tf_5m_eligible.set_index('timestamp')

# Create arm signals matrix (Matching Original)
arm_signals_df = pd.DataFrame({
    'S_top': tf_5m_signals['S_top'],
    'S_bot': tf_5m_signals['S_bot'],
    'S_mood': tf_5m_signals['S_mood'],
    'S_model': tf_5m_signals['S_model'],
}, index=tf_5m_signals.index)

arm_eligible_df = pd.DataFrame(False, index=tf_5m_signals.index, columns=['pros','amateurs','mood','model'])
arm_eligible_df['pros'] = tf_5m_eligible['pros']
arm_eligible_df['amateurs'] = tf_5m_eligible['amateurs']
arm_eligible_df['mood'] = tf_5m_eligible['mood']
arm_eligible_df['model'] = tf_5m_eligible['model']

# Export to ndarray for backtest (Matching Original)
arm_signals = arm_signals_df[['S_top','S_bot','S_mood','S_model']].values
arm_eligible = arm_eligible_df[['pros','amateurs','mood','model']].values
arm_names = ['pros','amateurs','mood','model']

print(f"Arm signals shape: {arm_signals.shape}")
print(f"Arm eligible shape: {arm_eligible.shape}")
print(f"Arm names: {arm_names}")

# Create ADV series (Matching Original)
adv20_by_ts = pd.Series(25000000.0, index=bt['timestamp'])  # $25M ADV
ANNUALIZER = np.sqrt(365*24*12)  # 5-minute bars

print("Signal generation complete!")


## 6. Backtesting Execution


In [ ]:
# Run Backtest (Matching Original)
print("Running backtest...")

# Prepare backtest data (Matching Original)
bt_df = bt.set_index('timestamp')
side_eps_vec = np.array([S_MIN, S_MIN, M_MIN, ALPHA_MIN], dtype=float)

# Run backtest (Matching Original)
Eq_df, Tr_df, Bu_df, met = run_allocator_backtest(
    bt_df=bt_df,
    arm_signals=arm_signals,
    arm_eligible=arm_eligible,
    adv_series=adv20_by_ts,
    cooldown_bars=COOLDOWN,
    cost_bps=COST_BP,
    impact_k=IMPACT_K,
    side_eps_vec=side_eps_vec,
    eps=1e-12,
)

print("Backtest Results:")
print(f"  Final Equity: {met['final_equity']:.4f}")
print(f"  Number of Trades: {met['n_trades']}")
print(f"  Sharpe Ratio: {met['sharpe']:.4f}")
print(f"  Sortino Ratio: {met['sortino']:.4f}")
print(f"  Max Drawdown: {met['maxDD']:.4f}")
print(f"  Turnover: {met['turnover']:.4f}")
print(f"  Hit Rate: {met['hit_rate']:.4f}")

# Display results
print(f"\nEquity DataFrame shape: {Eq_df.shape}")
print(f"Trades DataFrame shape: {Tr_df.shape}")
print(f"Bandit Updates DataFrame shape: {Bu_df.shape}")

# Show sample data
print(f"\nSample Equity Data:")
print(Eq_df.head())

print(f"\nSample Trades Data:")
print(Tr_df.head())

print(f"\nSample Bandit Updates:")
print(Bu_df.head())


## 7. Grid Search Optimization


In [ ]:
# Extended grid with cooldown and confidence sweep (Matching Original)
import itertools
from datetime import datetime

COOLDOWN_GRID = [1, 3, 5, 10]
CONF_MIN_GRID_EXT = [0.6, 0.7, 0.8]
FEE_BPS_GRID = [3.0, 5.0, 10.0]
SIGMA_TARGET_GRID = [0.10, 0.20, 0.30]

GATE_VARIANTS = [
    None,
    {'name': 'cons2_adv0', 'consensus_min': 2, 'advantage_min': 0.0},
    {'name': 'cons3_adv0', 'consensus_min': 3, 'advantage_min': 0.0},
    {'name': 'cons2_adv0p02', 'consensus_min': 2, 'advantage_min': 0.02},
]

print("Grid search parameters defined:")
print(f"  Cooldown grid: {COOLDOWN_GRID}")
print(f"  Confidence grid: {CONF_MIN_GRID_EXT}")
print(f"  Fee grid: {FEE_BPS_GRID}")
print(f"  Sigma target grid: {SIGMA_TARGET_GRID}")
print(f"  Gate variants: {len(GATE_VARIANTS)}")


In [ ]:
# Run Extended Grid Search (Matching Original)
def run_extended_grid():
    """Run extended grid search optimization (Matching Original)"""
    rows = []
    stamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    
    print(f"Starting extended grid search at {stamp}")
    
    # Grid search over all combinations
    for cooldown in COOLDOWN_GRID:
        for conf_min in CONF_MIN_GRID_EXT:
            for fee_bps in FEE_BPS_GRID:
                for sigma_target in SIGMA_TARGET_GRID:
                    for gate_cfg in GATE_VARIANTS:
                        
                        # Update global parameters
                        global COOLDOWN, CONF_MIN, COST_BP, SIGMA_TARGET
                        COOLDOWN = cooldown
                        CONF_MIN = conf_min
                        COST_BP = fee_bps
                        SIGMA_TARGET = sigma_target
                        
                        # Run backtest with current configuration
                        try:
                            # Prepare side thresholds
                            side_eps_vec = np.array([S_MIN, S_MIN, M_MIN, ALPHA_MIN], dtype=float)
                            
                            # Run backtest
                            Eq, Tr, Bu, metrics = run_allocator_backtest(
                                bt_df=bt_df,
                                arm_signals=arm_signals,
                                arm_eligible=arm_eligible,
                                adv_series=adv20_by_ts,
                                cooldown_bars=COOLDOWN,
                                cost_bps=COST_BP,
                                impact_k=IMPACT_K,
                                side_eps_vec=side_eps_vec,
                                eps=1e-12,
                            )
                            
                            # Create row
                            row = {
                                'COOLDOWN': cooldown,
                                'CONF_MIN': conf_min,
                                'FEE_BPS': fee_bps,
                                'SIGMA_TARGET': sigma_target,
                                'GATE': gate_cfg['name'] if gate_cfg else 'none',
                                'final_equity': metrics['final_equity'],
                                'n_trades': metrics['n_trades'],
                                'sharpe': metrics['sharpe'],
                                'sortino': metrics['sortino'],
                                'maxDD': metrics['maxDD'],
                                'turnover': metrics['turnover'],
                                'hit_rate': metrics['hit_rate'],
                            }
                            
                            rows.append(row)
                            
                        except Exception as e:
                            print(f"Error in grid search: {e}")
                            continue
    
    # Create results DataFrame
    results_df = pd.DataFrame(rows)
    
    print(f"Grid search complete: {len(results_df)} configurations tested")
    
    # Find best configuration
    if not results_df.empty:
        best_idx = results_df['sharpe'].idxmax()
        best_config = results_df.loc[best_idx]
        
        print(f"\nBest Configuration:")
        print(f"  Sharpe: {best_config['sharpe']:.4f}")
        print(f"  Cooldown: {best_config['COOLDOWN']}")
        print(f"  Confidence: {best_config['CONF_MIN']}")
        print(f"  Fee BPS: {best_config['FEE_BPS']}")
        print(f"  Sigma Target: {best_config['SIGMA_TARGET']}")
        print(f"  Gate: {best_config['GATE']}")
    
    return results_df

# Run grid search
grid_results = run_extended_grid()

# Display top 10 results
print(f"\nTop 10 Configurations by Sharpe Ratio:")
top_10 = grid_results.nlargest(10, 'sharpe')
print(top_10[['COOLDOWN', 'CONF_MIN', 'FEE_BPS', 'SIGMA_TARGET', 'GATE', 'sharpe', 'final_equity', 'n_trades']])


## 8. Results Analysis & Visualization


In [ ]:
# Results Analysis & Visualization (Matching Original)
print("Creating visualizations...")

# Set up plotting style
plt.style.use('default')
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Unified Overlay Trading Results', fontsize=16)

# 1. Equity Curve
axes[0, 0].plot(Eq_df.index, Eq_df['equity'])
axes[0, 0].set_title('Equity Curve')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Equity')
axes[0, 0].grid(True)

# 2. Drawdown
drawdown = (Eq_df['equity'] / Eq_df['equity'].cummax() - 1.0) * 100
axes[0, 1].fill_between(Eq_df.index, drawdown, 0, alpha=0.3, color='red')
axes[0, 1].set_title('Drawdown (%)')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Drawdown %')
axes[0, 1].grid(True)

# 3. Bandit Arm Selection
if not Bu_df.empty:
    arm_counts = Bu_df['chosen'].value_counts().sort_index()
    arm_names = ['pros', 'amateurs', 'mood', 'model']
    arm_labels = [arm_names[i] if i < len(arm_names) else f'arm_{i}' for i in arm_counts.index]
    
    axes[1, 0].bar(arm_labels, arm_counts.values)
    axes[1, 0].set_title('Bandit Arm Selection Count')
    axes[1, 0].set_xlabel('Arm')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Performance Metrics
metrics_data = {
    'Metric': ['Sharpe', 'Sortino', 'Max DD', 'Hit Rate'],
    'Value': [met['sharpe'], met['sortino'], met['maxDD'], met['hit_rate']]
}
metrics_df = pd.DataFrame(metrics_data)

axes[1, 1].bar(metrics_df['Metric'], metrics_df['Value'])
axes[1, 1].set_title('Performance Metrics')
axes[1, 1].set_ylabel('Value')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Additional Analysis
print(f"\nDetailed Performance Analysis:")
print(f"  Total Return: {(met['final_equity'] - 1.0) * 100:.2f}%")
print(f"  Annualized Return: {((met['final_equity'] ** (252*24*12/len(Eq_df))) - 1) * 100:.2f}%")
print(f"  Volatility: {Eq_df['equity'].pct_change().std() * np.sqrt(252*24*12) * 100:.2f}%")
print(f"  Max Drawdown: {met['maxDD'] * 100:.2f}%")
print(f"  Calmar Ratio: {met['sharpe'] / met['maxDD'] if met['maxDD'] > 0 else np.nan:.4f}")

# Trade Analysis
if not Tr_df.empty:
    print(f"\nTrade Analysis:")
    print(f"  Total Trades: {len(Tr_df)}")
    print(f"  Average Trade Size: {Tr_df['to_pos'].abs().mean():.4f}")
    print(f"  Trade Frequency: {len(Tr_df) / (len(Eq_df) / (252*24*12)):.2f} trades/day")

# Bandit Analysis
if not Bu_df.empty:
    print(f"\nBandit Analysis:")
    for i, arm_name in enumerate(arm_names):
        arm_rewards = Bu_df[Bu_df['chosen'] == i]['reward']
        if len(arm_rewards) > 0:
            print(f"  {arm_name}: {len(arm_rewards)} selections, avg reward: {arm_rewards.mean():.6f}")

print("Visualization complete!")


## 9. Export & Deployment


In [ ]:
# Export Results and Models (Matching Original)
print("Exporting results and models...")

# Create models directory
os.makedirs('models', exist_ok=True)

# Export trained models
timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')

# Save base models
for name, model in base_models.items():
    model_path = f'models/{name}_model_{timestamp}.joblib'
    joblib.dump(model, model_path)
    print(f"Saved {name} model to {model_path}")

# Save meta model
meta_model_path = f'models/meta_model_{timestamp}.joblib'
joblib.dump(meta_model, meta_model_path)
print(f"Saved meta model to {meta_model_path}")

# Save feature columns
feature_columns_path = f'models/feature_columns_{timestamp}.json'
with open(feature_columns_path, 'w') as f:
    json.dump(feature_columns, f)
print(f"Saved feature columns to {feature_columns_path}")

# Create model manifest
manifest = {
    'timestamp': timestamp,
    'model_type': 'unified_overlay',
    'base_models': list(base_models.keys()),
    'feature_columns': feature_columns,
    'overlay_timeframes': OVERLAY_TIMEFRAMES,
    'rollup_windows': ROLLUP_WINDOWS,
    'timeframe_weights': TIMEFRAME_WEIGHTS,
    'training_params': {
        'S_MIN': S_MIN,
        'M_MIN': M_MIN,
        'CONF_MIN': CONF_MIN,
        'ALPHA_MIN': ALPHA_MIN,
        'COOLDOWN': COOLDOWN,
        'COST_BP': COST_BP,
        'SIGMA_TARGET': SIGMA_TARGET,
    },
    'performance_metrics': met,
    'grid_search_results': grid_results.to_dict('records') if not grid_results.empty else []
}

manifest_path = f'models/manifest_{timestamp}.json'
with open(manifest_path, 'w') as f:
    json.dump(manifest, f, indent=2)
print(f"Saved model manifest to {manifest_path}")

# Export backtest results
results_dir = f'results_{timestamp}'
os.makedirs(results_dir, exist_ok=True)

# Save equity curve
Eq_df.to_csv(f'{results_dir}/equity_curve.csv')
print(f"Saved equity curve to {results_dir}/equity_curve.csv")

# Save trades
Tr_df.to_csv(f'{results_dir}/trades.csv', index=False)
print(f"Saved trades to {results_dir}/trades.csv")

# Save bandit updates
Bu_df.to_csv(f'{results_dir}/bandit_updates.csv', index=False)
print(f"Saved bandit updates to {results_dir}/bandit_updates.csv")

# Save grid search results
grid_results.to_csv(f'{results_dir}/grid_search_results.csv', index=False)
print(f"Saved grid search results to {results_dir}/grid_search_results.csv")

# Save overlay signals
overlay_signals.to_csv(f'{results_dir}/overlay_signals.csv', index=False)
print(f"Saved overlay signals to {results_dir}/overlay_signals.csv")

# Save eligibility matrix
eligibility_matrix.to_csv(f'{results_dir}/eligibility_matrix.csv', index=False)
print(f"Saved eligibility matrix to {results_dir}/eligibility_matrix.csv")

# Create summary report
summary_report = f"""
# Unified Overlay Trading Results Summary

## Model Training
- **Timestamp**: {timestamp}
- **Model Type**: Unified Overlay (Single model for all timeframes)
- **Base Models**: {', '.join(base_models.keys())}
- **Feature Columns**: {len(feature_columns)}
- **Overlay Timeframes**: {', '.join(OVERLAY_TIMEFRAMES)}

## Performance Metrics
- **Final Equity**: {met['final_equity']:.4f}
- **Sharpe Ratio**: {met['sharpe']:.4f}
- **Sortino Ratio**: {met['sortino']:.4f}
- **Max Drawdown**: {met['maxDD']:.4f}
- **Number of Trades**: {met['n_trades']}
- **Hit Rate**: {met['hit_rate']:.4f}

## Grid Search Results
- **Total Configurations Tested**: {len(grid_results)}
- **Best Sharpe Ratio**: {grid_results['sharpe'].max():.4f} if not grid_results.empty else 'N/A'

## Files Exported
- Models: models/
- Results: {results_dir}/
- Manifest: {manifest_path}

## Next Steps
1. Copy models to MetaStackerBandit/live_demo/models/
2. Update MetaStackerBandit configuration
3. Deploy overlay system
4. Monitor performance
"""

with open(f'{results_dir}/summary_report.md', 'w') as f:
    f.write(summary_report)

print(f"Saved summary report to {results_dir}/summary_report.md")

print("\nExport complete!")
print(f"All files saved with timestamp: {timestamp}")
print(f"Results directory: {results_dir}")
print(f"Models directory: models/")

# Display final summary
print(f"\n=== FINAL SUMMARY ===")
print(f"Unified Overlay Architecture Implementation Complete!")
print(f"Single model trained on 5m data for all timeframes")
print(f"Performance: Sharpe={met['sharpe']:.4f}, MaxDD={met['maxDD']:.4f}")
print(f"Ready for deployment to MetaStackerBandit")
